# Global effect

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt




import pandas as pd

with open('/home/23m1526/ua.base', 'r') as file:
    # Read the lines of the file
    lines = file.readlines()


data = []
for line in lines:
    parts = line.strip().split('\t')
    user_id = int(parts[0])
    item_id = int(parts[1])
    rating = int(parts[2])

    data.append((user_id, item_id, rating))


df = pd.DataFrame(data, columns=['userId', 'movieId', 'rating'])

# Print the first few rows of the DataFrame
df


df['movieId'].max()
df['movieId'].min()
temp = 1
userItem  = np.zeros((df['userId'].max()+1,df['movieId'].max()+1))
print(userItem )
print(userItem.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(944, 1683)


In [3]:
for i in range (len(df['userId'])):
  userItem[df.userId[i]][df.movieId[i]] = df.rating[i]


In [48]:
R = userItem[1:]

R = R[:, 1:]

ep = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
print(R)

[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]


In [49]:

import math
def RMSE(actual, predicted):
    err = 0
    
    count = np.count_nonzero(actual)
    for i in range(actual.shape[0]):
        for j in range(actual.shape[1]):
            if actual[i][j] > 0:
                
                err += (actual[i][j] - predicted[i][j])**2
    
    if count == 0: "error is not calculated therefore count = 0 "
    else : 
        mse = err/count
    rmse = math.sqrt(mse)
    
    return  rmse
    

In [50]:
def evaluate_global_average(R, epsilon1):
    # Calculate global average
    global_avg = (np.sum(R) + np.random.laplace(5-1/epsilon1)) / 1e5
    

    return global_avg


In [73]:
GAvg =[] 
    
for ep in range(10):
    G_avg = evaluate_global_average(R, 0.2)  # Compute global average for current epsilon      
    rmse_temp = RMSE(R, G)  # Now G is a matrix of the same shape as R
    GAvg.append(rmse_temp)
print(GAvg)

[0.9293250136309656, 0.9293250136309656, 0.9293250136309656, 0.9293250136309656, 0.9293250136309656, 0.9293250136309656, 0.9293250136309656, 0.9293250136309656, 0.9293250136309656, 0.9293250136309656]


In [51]:
import numpy as np



def evaluate_item_averages(R,A, beta_i, epsilon1, epsilon2 ):
    
    r_min, r_max = 1,5
    # Calculate global average
    global_avg = (np.sum(R) + np.random.laplace(5-1/epsilon1)) / 1e5

    # Initialize item averages
    item_averages = np.zeros(R.shape[1])
    
    Rk = np.sum(A, axis=0) # no of rating given to a item    
 
    for j in range(R.shape[1]):
        # Select ratings for item j
        Rj = R[:, j]

        # Calculate item average
        item_avg = (np.sum(Rj) + beta_i * global_avg + np.random.laplace(scale= 4 /epsilon2)) / (Rk[j] + beta_i)

        # Clamp item average to specified range
        item_avg = np.clip(item_avg, 1, 5 )

        item_averages[j] = item_avg

    return item_averages


In [52]:


A = R>0.5; A[A == True] = 1; A[A == False]   = 0; A = A.astype(np.float64, copy=False)
print(A)
ItemAvg = evaluate_item_averages(R,A,  10,0.1, 0.5)
Rj = np.sum(A, axis=1) # no of rating given to a item

[[1. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [75]:

IA = []
ep = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

    
    
for ep in ep:
    Item_avg = evaluate_item_averages(R, A,10,0.2, ep)  # Compute global average for current epsilon
      # Create a matrix filled with the global average
    A = np.tile(np.array(Item_avg), (R.shape[0], 1))
        
    rmse_temp = RMSE(R, A)  # Now G is a matrix of the same shape as R
    IA.append(rmse_temp)


print('The RMSE for different epsilons are', IA)

The RMSE for different epsilons are [2.702699999994313, 2.761327700579407, 2.7032811449987464, 2.7608698457273393, 2.703850037626697, 2.7604239119295984, 2.704400423625238, 2.759987872345209, 2.704919363901997, 2.759579037621829]


In [58]:
import numpy as np

def evaluate_user_averages(R,A, item_avgs, beta_u, epsilon_1, epsilon_2):
    Ri = np.sum(A, axis=1)# no of rating given by a user    
    
    R0 = R - item_avgs

    R2 = np.zeros_like(R0)
     
#Assign values from R0 where R1 is 1
    R2[A == 1] = R0[A == 1]
    R0 = R2

    n_users, m_items = R.shape
    # Calculate the new global average based on adjusted ratings
    global_avg_0 = np.mean(R0) + np.random.laplace(scale=2 * np.max(np.abs(R)) / epsilon_1)/(10**5 )
    
    
    
    user_avgs = np.zeros(n_users)

    for v in range(n_users):
        # Select ratings for user v
        ratings_v = R0[v, :]
        # Calculate user average
   
        # Add fictitious ratings with the new global average for each user
        user_avg = (np.sum(ratings_v) + beta_u * global_avg_0 + np.random.laplace(scale=4 / epsilon_2))/(Ri[v] + beta_u)
        # Clamp user average to [-2, 2]
        user_avgs[v] = np.clip(user_avg, -2, 2)

    return user_avgs

In [59]:
UserAvg = evaluate_user_averages(R,A, ItemAvg, 20, 0.2, 2)
print(UserAvg)

[ 8.52504845e-02  1.04455368e-01 -2.52348896e-01  3.56297542e-01
 -3.18632453e-01 -1.82797490e-01  4.33830956e-01  1.93724711e-01
  1.20594028e-01  3.67874982e-01 -6.70368348e-02  3.32913056e-01
 -1.94530790e-01  2.35706852e-01 -3.35101652e-01  5.77550196e-01
 -2.42053717e-01  1.14882338e-01  2.94135297e-02 -3.10379751e-01
 -2.37530383e-01  6.59091957e-03 -1.02888065e-01  4.43146191e-01
  1.33522786e-01 -3.40052428e-01 -5.70630316e-05  1.43835901e-01
  1.49234021e-01  1.97668722e-01  9.21138766e-02 -1.94174258e-01
  1.01787018e-01  1.33391759e-01 -1.33781899e-02  4.48381176e-01
  1.92339110e-01  5.10607638e-01  1.97035522e-01 -4.48177365e-01
 -1.34986539e-01  1.99799207e-01  1.59550543e-01  3.64132794e-02
  2.26732482e-01  2.18810094e-01  1.75546087e-02 -1.08737059e-01
 -6.17483824e-01  5.53040803e-02 -1.53870458e-01  3.74896928e-01
  2.30922977e-01  1.70364759e-01  1.06699746e-01  1.79078878e-01
  1.61229539e-01  9.56580762e-02  3.18751487e-01  2.84458937e-01
 -2.73053377e-01 -2.27136

In [60]:
def rating_globle_effect(U,I):
  ratings = U[:, np.newaxis] + I
  return ratings
GAvg = rating_globle_effect(UserAvg,ItemAvg)
print(GAvg)


[[3.95356977 3.277827   3.13395543 ... 1.08525048 1.08525048 1.80209348]
 [3.97277466 3.29703189 3.15316031 ... 1.10445537 1.10445537 1.82129836]
 [3.61597039 2.94022762 2.79635604 ... 0.7476511  0.7476511  1.46449409]
 ...
 [3.96135326 3.28561049 3.14173891 ... 1.09303397 1.09303397 1.80987696]
 [4.2939852  3.61824243 3.47437085 ... 1.42566591 1.42566591 2.1425089 ]
 [3.8651955  3.18945273 3.04558115 ... 0.99687621 0.99687621 1.7137192 ]]


In [61]:
GEff_Rmse = []
ep = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
for ep in (ep):
    I = evaluate_item_averages(R,A ,20,0.2, ep)
    U = evaluate_user_averages(R,A, I,29,0.2,ep)
    G = rating_globle_effect( U ,I)      
    temp = RMSE(R,G)
    GEff_Rmse.append(temp)

In [62]:
GEff_Rmse

[0.9300801007487787,
 0.9295197114847286,
 0.9294149105127584,
 0.9292767133232992,
 0.9293774990886291,
 0.9293113554962521,
 0.9293095339223715,
 0.9293381253316699,
 0.9293088879051371,
 0.9293250136309656]

In [63]:
import csv



csv_file_path = 'output2.csv'

In [64]:
with open(csv_file_path, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)

    writer.writerow(['GEff_Rmse'] + GEff_Rmse)

In [65]:
def input_perterbation_sgd(R,A, d, gamma, lambd, epsilon):
    k = 60
    num_users, num_items = R.shape
    P = np.random.rand(num_users, d)
    Q = np.random.rand(num_items, d)
    R1 = R + np.random.laplace(scale=(4) / epsilon, size=R.shape)
    R0 = np.zeros((R.shape))
    B =1
    for i in range(R.shape[0]):
        for j in range (R.shape[1]): 
            if A[i,j] != 0: R0[i,j ] = R1[i,j]

    
    R0 = np.clip(R0, -B, B)
    # Assuming R is a numpy array, find non-zero (i.e., rated) entries
    rated_indices = np.argwhere(R > 0)

    for epoch in range(k):
        np.random.shuffle(rated_indices)  # Shuffle the observed ratings
        for i, j in rated_indices:
            e_ui = R[i, j] - np.dot(P[i], Q[j]) 
       

            # Update rule remains the same, but only applied to observed ratings
            P[i] += gamma * (e_ui * Q[j] - lambd * P[i])
            Q[j] += gamma * (e_ui * P[i] - lambd * Q[j])

    return P, Q , P.dot(Q.T)


In [66]:
p,q , r = input_perterbation_sgd(R,A, 3,0.006,0.06,  epsilon = 2  )

In [67]:
import math

    
l =RMSE(R,r) 
print(l)

0.8561726879930089


In [69]:
ISGD_Rmse = []
ep = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
for _ in (ep):

    p,q , r = input_perterbation_sgd(R,A, 3,0.006,0.06,  epsilon = _  )

    temp = RMSE(R,r)
    ISGD_Rmse.append(temp)

In [71]:
print(ISGD_Rmse)

[0.8569289430625157, 0.8655257168357734, 0.8580136584651753, 0.8544487006712539, 0.8562101858948241, 0.855242233438414, 0.8593857589149573, 0.8555173932052811, 0.8571501580831401, 0.8608188658363182]


In [76]:
PGEff_Rmse = []
ep = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
for ep in (ep):
    I = evaluate_item_averages(R,A ,20,0.2, ep)
    U = evaluate_user_averages(R,A, I,10,0.2,ep)
    G = rating_globle_effect( U ,I) 
    p,q , r = input_perterbation_sgd(R,A, 3,0.006,0.06,  epsilon = _  )
    G += r
    temp = RMSE(R,G)
    PGEff_Rmse.append(temp)
    
    
    


In [77]:
print(PGEff_Rmse)

[0.9760241740536227, 0.9740574769328367, 0.9708401899226916, 0.9718834064974764, 0.9773660484637611, 0.9741685800799625, 0.9728852362064367, 0.9697117788415919, 0.97337816527578, 0.9663711982136984]


In [129]:
earlier_ISGD_RMSE  = [0.9674526648314522, 0.96833593068766, 0.9644386770392005, 0.9718298386316295, 
 0.970845225042154, 0.9672090854896878, 0.963507505096101, 0.9674732279118728, 0.969613924973632, 0.9693952362699685]

In [ ]:
def input_perterbation_mf(R, k, epsilon ,n_epoch=500, lr=.0003, l2 = 0.06):

In [141]:
p,q , r = input_perterbation_sgd(R,A, 3,0.006,0.06,  epsilon = 2  )

In [142]:
print(r)

[[3.90847536 3.09509205 3.00867044 ... 1.80236077 3.34318911 3.22104645]
 [3.99105761 3.07472431 2.44605352 ... 1.61686061 3.52685085 3.03051305]
 [3.27365775 2.69894709 2.80912499 ... 1.58987727 2.70215979 2.79324885]
 ...
 [4.17301759 3.32885527 3.09696903 ... 1.86942029 3.56285729 3.37704924]
 [4.38161953 3.56830793 2.47777334 ... 1.62507015 3.75808577 3.1619705 ]
 [3.6382382  3.28082892 3.84768667 ... 1.96364231 2.74757714 3.33873964]]


In [138]:
I = evaluate_item_averages(R,A ,20,0.2, 2)
U = evaluate_user_averages(R,A, I,15,0.2,2)
G = rating_globle_effect( U ,I)  

In [139]:
print(G)

[[3.90353301 3.26243711 3.17620697 ... 3.34772873 3.33267955 2.95955464]
 [3.92969473 3.28859883 3.20236869 ... 3.37389045 3.35884127 2.98571636]
 [3.48041823 2.83932233 2.75309219 ... 2.92461395 2.90956477 2.53643987]
 ...
 [4.05184116 3.41074526 3.32451512 ... 3.49603688 3.4809877  3.10786279]
 [4.32519774 3.68410184 3.5978717  ... 3.76939346 3.75434428 3.38121937]
 [3.75195885 3.11086295 3.02463281 ... 3.19615457 3.18110539 2.80798049]]
